# Chapter 9 - Searching Data Structures and Finding Shortest Paths

This notebook contains code accompanying Chapter 9 Searching Data Structures and Finding Shortest Paths in *Practical Discrete Mathematics* by Ryan T. White and Archana Tikayat Ray

For most of the code in the chapter, we need to import the `NumPy` library.

In [10]:
import numpy

## A Python Implementation of DFS

In [11]:
# Depth First Search
#
# INPUTS
# A - an adjacency matrix. It should be square, symmetric, and binary
# source - the number of the source vertex
#
# OUTPUTS
# vertexList - an ordered list of vertices found in the search

def DFS(A, source):
    # reduce the source by 1 to avoid off-by-1 errors
    source -= 1

    # find the number of vertices
    n = A.shape[0]

    # initialize the unvisited vertex set to be full
    unvisited = [1] * n

    # initialize a queue with the source vertex
    stack = [source]

    # initialize the vertex list
    vertexList = []

    # while the stack is not empty
    while stack:
        # remove the just-visited vertex from the stack and store it
        v = stack.pop()

        # if v is unvisited, add it to our list and mark it as visited
        if unvisited[v]:
            # save and print the number of the newly visited vertex
            vertexList.append(v)

            # mark the vertex as visited
            unvisited[v] = 0

        # iterate through the vertices
        for u in range(n - 1, 0, -1):
            # add each unvisited neighbor to the stack
            if A[v,u] == 1 and unvisited[u] == 1:
                stack.append(u)

    # return the vertex list found by DFS
    return vertexList

Let's save the adjacency matrix for the graph in Figure 9.1.

In [12]:
# Save the adjacency matrix for the graph in Figure 9.1
A = numpy.array([[0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0],
                  [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                  [0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0],
                  [0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0],
                  [1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0],
                  [0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0],
                  [0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0],
                  [0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0],
                  [0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0],
                  [0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0],
                  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

Next, let's run DFS on the graph starting with source vertex 1.

In [13]:
# Run DFS on the graph with adjacency matrix A and source 1
vertexList = DFS(A,1)

# Add 1 to the vertex numbers
[x + 1 for x in vertexList]

[1, 2, 5, 4, 3, 6, 7, 8, 9, 10]

## Shortest Paths on Networks

The following function checks if a path exists between vertex i and vertex j.

In [14]:
# create a function that returns True if vertex i and vertex j are
# connected in the graph represented by the input adjacency matrix A
def isConnected(A, i, j):
    # initialize the paths matrix to adjacency matrix A
    paths = A

    # find the number of vertices in the graph
    numberOfVertices = A.shape[0]

    # find the number of edges in the graph
    numberOfEdges = numpy.sum(A)/2

    # if vi and vj are adjacent, return True
    if paths[i-1][j-1] > 0:
        print('Vertex', i, 'and vertex', j, 'are adjacent')
        return True

    else:
        # run the loop until we find a path
        for pathLength in range(2, numberOfVertices):
            # exponentiate the adjacency matrix
            paths = numpy.dot(paths, A)

            # if the element in row i, column j is more than 0, we
            # found a path
            if paths[i-1][j-1] > 0:
                print('There is a path with', pathLength,
                      'edges from vertex', i, 'to vertex', j)
                return True

        # found no paths, the vertices are not connected
        if pathLength == numberOfEdges:
            print('There are no paths from vertex', i, 'to vertex', j)
            return False

Let's create adjacency matrices for the graphs in Figure 9.6 and test our function.

In [16]:
# create an adjacency matrix for the graph G1
A1 = numpy.array([[0, 1, 1, 0, 1, 0], [1, 0, 1, 1, 0, 1],
                  [1, 1, 0, 1, 1, 0], [0, 1, 1, 0, 1, 0],
                  [1, 0, 1, 1, 0, 0], [0, 1, 0, 0, 0, 0]])

# check if various vertices are connected
print(isConnected(A1, 1, 4))
print(isConnected(A1, 2, 3))
print(isConnected(A1, 5, 6))

# create an adjacency matrix for graph G2
A2 = numpy.array([[0, 1, 0, 0, 0, 0], [1, 0, 0, 0, 0, 1],
                  [0, 0, 0, 1, 1, 0], [0, 0, 1, 0, 1, 0],
                  [0, 0, 1, 1, 0, 0], [0, 1, 0, 0, 0, 0]])

print(isConnected(A2, 1, 6))
print(isConnected(A2, 2, 5))
print(isConnected(A2, 1, 4))

There is a path with 2 edges from vertex 1 to vertex 4
True
Vertex 2 and vertex 3 are adjacent
True
There is a path with 3 edges from vertex 5 to vertex 6
True
There is a path with 2 edges from vertex 1 to vertex 6
True
There are no paths from vertex 2 to vertex 5
False
There are no paths from vertex 1 to vertex 4
False


## Python Implementation of Dijkstra’s Algorithm



In [17]:
import numpy

# Dijkstra's algorithm for finding shortest paths from the source
# vertex to all other vertices in the graph
#
# INPUTS
# W - a weight matrix. It should be a square matrix
# i - the number of the source node
#
# OUTPUTS
# shortestDistances - the shortest distances from the source to each
# vertex
# previousVertices - the previous vertex to the destination in a
# shortest path from the source to a destination
def Dijkstra(W, i):
    # find the number of vertices
    n = W.shape[0]

    # initialize the shortest distances to infinity
    shortestDistances = numpy.array([numpy.inf] * n)

    # initialize the previous vertices
    previousVertices = numpy.array([numpy.inf] * n)

    # initialize the unvisited vertex set to be full
    unvisited = numpy.array([1] * n)

    # mark the source as visited
    unvisited[i - 1] = 0

    # initialize distance from the source to the source as 0
    shortestDistances[i - 1] = 0

    # loop for iteration per vertex until the unvisited set is empty
    for _ in range(n):
        # find the distances to all unvisited adjacent vertices and
        # set others to 0
        distances = shortestDistances * unvisited

        # find the index of the nearest unvisited vertex (where
        # distances > 0)
        x = numpy.argmin(numpy.ma.masked_where(
            distances == 0, distances))

        # mark vertex x as visited
        unvisited[x] = 0

        # iterate through the vertices
        for v in range(n):

            oldDistance = shortestDistances[v]
            newDistance = shortestDistances[x] + W[v,x]
            adjacent = W[v,x] > 0
            unvis = unvisited[v]

            # if v and x are connected, v has not been visited, and we
            # find a shorter distance to node v...
            if adjacent and unvis and oldDistance > newDistance:
                # save the shortest distance found so far
                shortestDistances[v] = shortestDistances[x] + W[v,x]

                # save the previous vertex
                previousVertices[v] = x

    # print the table similar to the book
    print(numpy.array([numpy.arange(n) + 1, shortestDistances,
                       previousVertices + 1]).T)
    # return the outputs
    return shortestDistances, previousVertices


Let's create the weight matrix for the network in Figure 9.15 and run Dijkstra's algorithms to find the shortest paths from $v_1$ to all other vertices.

In [18]:
# Create a weight matrix for the network in Figure 9.15
W1 = numpy.array([[0, 4, 1, 0, 2, 0],
                  [4, 0, 2, 1, 0, 1],
                  [1, 2, 0, 1, 1, 0],
                  [0, 1, 1, 0, 2, 0],
                  [2, 0, 1, 2, 0, 0],
                  [0, 1, 0, 0, 0, 0]])

# Run Dijkstra's algorithm with a source at vertex v1
shortestDistances, previousVertices = Dijkstra(W1, 1)

[[ 1.  0. inf]
 [ 2.  3.  3.]
 [ 3.  1.  1.]
 [ 4.  2.  3.]
 [ 5.  2.  1.]
 [ 6.  4.  2.]]


Next, we write a function to clean up the outputs and display the paths

In [19]:
# Use the previousVertices chart to construct the shortest path from
# input source to input destination and print a string showing
# showing the path

def printShortestPath(shortestDistances, previousVertices, source,
                      destination):
    # avoid off-by-one error
    source -= 1
    destination -= 1

    # convert previousVertices to integers
    previousVertices = previousVertices.astype(int)

    # initialize the path with the destination
    path = [destination]

    # add the previous vertex from previousVertices until we reach
    # the source
    for _ in range(previousVertices.shape[0] - 1):
        # if the source is in the path, stop
        if path[-1] == source:
            break
        # if the source is not in the path, add the previous vertex
        else:
            path.append(previousVertices[path[-1]])

    # initialize an output string
    output = []

    # iterate through the path backwards (source to destination)
    for i in numpy.flip(path):
        # construct a list of strings to output
        if i > 0:
            output.append('->')

        output.append('v' + str(i + 1))

    # print the strings with no spaces
    print('Path =', *output, '\t\t Distance =',
          shortestDistances[destination])

And, we run it to find:

In [20]:
for i in range(2,7):
    printShortestPath(shortestDistances, previousVertices, 1, i)

Path = v1 -> v3 -> v2 		 Distance = 3.0
Path = v1 -> v3 		 Distance = 1.0
Path = v1 -> v3 -> v4 		 Distance = 2.0
Path = v1 -> v5 		 Distance = 2.0
Path = v1 -> v3 -> v2 -> v6 		 Distance = 4.0


<ipython-input-19-8470c64236d8>:12: RuntimeWarning: invalid value encountered in cast
  previousVertices = previousVertices.astype(int)


Next, we try a network that we know is not connected. First, we will check if the vertices in question are connected.

In [21]:
# find the shortest paths to connected vertices
def distancesWithinComponent(source):
    # initialize the connected component
    component = [source]

    # construct the connected component
    for i in range(1, W2.shape[0] + 1):
        if i != source and isConnected(W2, source, i):
            component.append(i)

    # find the weight matrix correponding to the connected component
    subnetwork = W2[numpy.array(component) - 1,:][:,numpy.array(component) - 1]

    # run Dijkstra's algorithm
    return Dijkstra(subnetwork, 1)



Let's find the paths from $v_1$.

In [22]:
# Create a weight matrix for the network in Figure 9.16
W2 = numpy.array([[0, 4, 0, 0, 0, 0],
                  [4, 0, 0, 0, 0, 1],
                  [0, 0, 0, 1, 4, 0],
                  [0, 0, 1, 0, 2, 0],
                  [0, 0, 4, 2, 0, 0],
                  [0, 1, 0, 0, 0, 0]])

distancesWithinComponent(1)

Vertex 1 and vertex 2 are adjacent
There is a path with 2 edges from vertex 1 to vertex 6
[[ 1.  0. inf]
 [ 2.  4.  1.]
 [ 3.  5.  2.]]


(array([0., 4., 5.]), array([inf,  0.,  1.]))

Let's find the paths from $v_3$.

In [23]:
distancesWithinComponent(3)

Vertex 3 and vertex 4 are adjacent
Vertex 3 and vertex 5 are adjacent
[[ 1.  0. inf]
 [ 2.  1.  1.]
 [ 3.  3.  2.]]


(array([0., 1., 3.]), array([inf,  0.,  1.]))

In [24]:
W3 =numpy.array([[0,1,0,0,0,7,8],
               [1,0,4,0,0,3,0],
               [0,4,0,5,7,2,0],
               [0,0,5,0,2,0,0],
               [0,0,7,2,0,0,0],
               [7,3,2,0,0,0,1],
               [8,0,0,0,0,1,0]])

shortestDistances, previousVertices = Dijkstra(W3, 1)

[[ 1.  0. inf]
 [ 2.  1.  1.]
 [ 3.  5.  2.]
 [ 4. 10.  3.]
 [ 5. 12.  3.]
 [ 6.  4.  2.]
 [ 7.  5.  6.]]


In [25]:
for i in range(2,7):
    printShortestPath(shortestDistances, previousVertices, 1, i)

Path = v1 -> v2 		 Distance = 1.0
Path = v1 -> v2 -> v3 		 Distance = 5.0
Path = v1 -> v2 -> v3 -> v4 		 Distance = 10.0
Path = v1 -> v2 -> v3 -> v5 		 Distance = 12.0
Path = v1 -> v2 -> v6 		 Distance = 4.0


<ipython-input-19-8470c64236d8>:12: RuntimeWarning: invalid value encountered in cast
  previousVertices = previousVertices.astype(int)
